# Exercise 6 - Solution
***

<div class=obj>
    <b>Aim:</b> To use a Monte Carlo methods to propagate error through a calculation.
</div>

<p></p>

As a reminder, the equations we are trying to use:

\begin{align}
T(\mathrm{K}) = \frac{10^4}{0.575(0.162) + 0.884(0.043)Y_\mathrm{Cr} - 0.897(0.025)\ln(\mathrm{k_d})},
\end{align}

\begin{align}
Y_\mathrm{Cr} = \frac{X_\mathrm{Cr}}{(X_\mathrm{Cr}+X_\mathrm{Al})},
\end{align}

\begin{align}
X_i = \frac{w_i/m_i}{\sum{w_j/m_j}},
\end{align}

\begin{align}
\mathrm{k_d} = \frac{\mathsf{Al_2O_3^{olivine}}}{\mathsf{Al_2O_3^{spinel}}},
\end{align}

|molecule| mean molecular mass |
|-----------|------------|
|SiO$_2$    | 60.0 |
|MgO        | 40.3 |
|AlO$_{1.5}$| 51.0 |
|TiO$_2$    | 79.9 |
|CaO        | 56.0 |
|FeO        | 71.8 |
|CrO$_{1.5}$| 76.0 |
|MnO        | 70.9 |
|NiO        | 74.7 |

All the hard work with this exercise is in manipulating the data.

In [1]:
import pandas as pd
import numpy as np

In [3]:
#readin data
df = pd.read_excel('data/crystal_ts.xlsx', index_col=None, na_values="NaN", header=0)
dfm = pd.read_csv('data/molecular_weights.txt', sep=' ', index_col=0, header=None)
df

,Location,Sample,sio2,sio2_1sig,mgo,mgo_1sig,al2o3,al2o3_1sig,tio2,tio2_1sig,cao,cao_1sig,feo,feo_1sig,cr2o3,cr2o3_1sig,mno,mno_1sig,nio,nio_1sig
0,Hawaii,olivine,40.94770,2.866339,47.598800,3.331916,0.049200,0.003444,0.261150,0.018281,11.24145,0.786902,0.107100,0.007497,NaN,NaN,0.15985,0.011189,0.413700,0.028959
1,Hawaii,spinel,0.11910,0.008337,13.379233,0.936546,14.047633,0.983334,1.723467,0.120643,NaN,NaN,21.915167,1.534062,46.478867,3.253521,0.20150,0.014105,0.243033,0.017012
2,Iceland,olivine,40.85429,2.859800,47.811850,3.346830,0.083720,0.005860,NaN,NaN,0.41181,0.028827,9.920710,0.694450,NaN,NaN,NaN,NaN,0.297990,0.020859
3,Iceland,spinel,NaN,NaN,18.938300,1.325681,43.656700,3.055969,NaN,NaN,NaN,NaN,13.470400,0.942928,21.388600,1.497202,NaN,NaN,NaN,NaN


In [10]:
#create a dictionary to store the atomic fraction 
spHawaii = {}
spIceland = {}
#load the dictionary with zeros
for e in dfm.index:
    spHawaii[e] = 0 
    spIceland[e] = 0
    #df.sio2[(df.Location == 'Hawaii') & (df.Sample == 'spinel')].isna()/dfm

#the files have different names for the oxides reflecting whether, e.g., Al2O3 is being talked about or alo1.5
# we deal with this by manually constructing two ordered lists than can be used to access the relevant elements of the data files
#oxide weight fractions...
wvec = ['sio2', 'mgo', 'al2o3', 'tio2', 'feo', 'cr2o3', 'mno', 'nio']
#...and the names of the error columns, which we use list comprehension to produce from the previous list
wvece = [w + '_1sig' for w in wvec]
#cation mole fractions 
mvec = ['sio', 'mgo', 'alo', 'tio', 'feo', 'cro', 'mno', 'nio']

#number of iterations to perform for Monte Carlo error propagation 
Ndraw = 1000

#setup coefficients for temperature calculation from Coogan et al. 2014
coeffs = np.array([0.575, 0.884, 0.897])
coeffse = np.array([0.162, 0.043, 0.025])

#setup dataframe to hold temporary data generated by random perturbation according to error
dftIce = pd.DataFrame()
dftHwi = pd.DataFrame()

#setup to arrays to hold calculated temperatures, the length 
TIce = np.zeros(Ndraw)
THwi = np.zeros(Ndraw)

for p in range(0, Ndraw):

    #generate randomly perturbed data
    dftIce = pd.DataFrame(index=['spinel', 'olivine'],columns=wvec)
    dftHwi = pd.DataFrame(index=['spinel', 'olivine'],columns=wvec)
    for i, w in enumerate(wvec):
        inc = (df.Location == 'Iceland') & (df.Sample == 'spinel')
        dftIce.loc['spinel'][w] = np.random.normal(df[w][inc], df[wvece[i]][inc])[0]
        inc = (df.Location == 'Iceland') & (df.Sample == 'olivine')
        dftIce.loc['olivine'][w] = np.random.normal(df[w][inc], df[wvece[i]][inc])[0]

        inc = (df.Location == 'Hawaii') & (df.Sample == 'spinel')
        dftHwi.loc['spinel'][w] = np.random.normal(df[w][inc], df[wvece[i]][inc])[0]
        inc = (df.Location == 'Hawaii') & (df.Sample == 'olivine')
        dftHwi.loc['olivine'][w] = np.random.normal(df[w][inc], df[wvece[i]][inc])[0]

   
    loc = 'Hawaii'
    for i, e in enumerate(wvec):
        spHawaii[mvec[i]] = np.array(df[e][(df.Location == loc) & (df.Sample == 'spinel')])[0]/np.array(dfm.loc[mvec[i]])[0]
    loc = 'Iceland'
    for i, e in enumerate(wvec):
        spIceland[mvec[i]] = np.array(df[e][(df.Location == loc) & (df.Sample == 'spinel')])[0]/np.array(dfm.loc[mvec[i]])[0]

    #calculate the cation fraction
    ssH = 0
    ssI = 0
    for s in spHawaii.keys():
        if ~ np.isnan(spHawaii[s]) and spHawaii[s] != 0:
            ssH = ssH + spHawaii[s]
        if ~ np.isnan(spIceland[s]) and spIceland[s] != 0:
            ssI = ssI + spIceland[s]
    for s in spHawaii.keys():
        if ~ np.isnan(spHawaii[s]) and spHawaii[s] != 0:
            spHawaii[s] = spHawaii[s]/ssH
        if ~ np.isnan(spIceland[s]) and spIceland[s] != 0:
            spIceland[s] = spIceland[s]/ssI

    # randomly perturb parameterisation parameters
    coeffst = np.random.normal(coeffs, coeffse)

    Y = spIceland['cro']/(spIceland['cro'] + spIceland['alo'])
    kd = dftIce.loc['olivine'].al2o3/dftIce.loc['spinel'].al2o3
    TIce[p] = 10**4/(coeffs[0] + coeffs[1]*Y - coeffs[2]*np.log(kd))

    Y = spHawaii['cro']/(spHawaii['cro'] + spHawaii['alo'])
    kd = dftHwi.loc['olivine'].al2o3/dftHwi.loc['spinel'].al2o3
    THwi[p] = 10**4/(coeffs[0] + coeffs[1]*Y - coeffs[2]*np.log(kd))

print('Iceland', '{:.1f}'.format(np.mean(TIce)), '+/-', '{:.1f}'.format(np.std(TIce)))
print('Hawaii', '{:.1f}'.format(np.mean(THwi)), '+/-', '{:.1f}'.format(np.std(THwi)))

Iceland 1560.6 +/- 21.7
Hawaii 1598.5 +/- 23.0
